In [12]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve
from sagemaker.s3 import *
import sys

if int(sagemaker.__version__.split('.')[0]) == 2:
    !{sys.executable} -m pip install sagemaker==1.72.0
    print("Installing previous SageMaker Version. Please restart the kernel")
else:
    print("Version is good")

role = get_execution_role()
sess = sagemaker.Session(default_bucket=None)
region = boto3.session.Session().region_name
print("Region = {}".format(region))
sm = boto3.Session().client('sagemaker')


Installing previous SageMaker Version. Please restart the kernel
Region = us-east-1


In [35]:

# see https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html for which inputs to use
# see https://github.com/aws/deep-learning-containers/blob/master/available_images.md for registry paths with custom algorithms


training_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04"
print (training_image)

763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04


In [36]:
!pip install sagemaker-experiments 
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [38]:
rawbucket= sess.default_bucket() # Alternatively you can use our custom bucket here. 
data_bucket = "treetracker-training-images"
prefix = 'sagemaker-modelmonitor' # use this prefix to store all files pertaining to this workshop.

dataprefix = prefix + '/data'
traindataprefix = prefix + '/train_data'
testdataprefix = prefix + '/test_data'
testdatanolabelprefix = prefix + '/test_data_no_label'
trainheaderprefix = prefix + '/train_headers'

s3_train_key = "imnet/"
s3_validation_key = ""
s3_train = 's3://{}/{}/'.format(data_bucket, s3_train_key)
s3_validation = 's3://{}/{}/'.format(data_bucket, s3_validation_key)
sagemaker_train = 's3://{}/{}/'.format(rawbucket, s3_train_key)
sagemaker_validation = 's3://{}/{}/'.format(rawbucket, s3_validation_key)


In [ ]:
%%writefile preprocessing.py


In [39]:
# Copy the preprocessing code over to the s3 bucket
codeprefix = prefix + '/code'
codeupload = sess.upload_data('preprocessing.py', bucket=rawbucket, key_prefix=codeprefix)
print(codeupload)

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessing.py'

In [40]:
train_data_location = rawbucket + '/' + traindataprefix
test_data_location = rawbucket+'/'+testdataprefix
print("Training data location = {}".format(train_data_location))
print("Test data location = {}".format(test_data_location))

Training data location = sagemaker-us-east-1-683584525758/sagemaker-modelmonitor/train_data
Test data location = sagemaker-us-east-1-683584525758/sagemaker-modelmonitor/test_data


In [41]:

from sagemaker.processing import ProcessingInput, ProcessingOutput

pre_input = ProcessingInput(source=s3_train, destination=None, input_name="pre")
pre_output = ProcessingOutput(source=None, destination=sagemaker_train)

